<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#无诱导" data-toc-modified-id="无诱导-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>无诱导</a></span></li><li><span><a href="#有诱导" data-toc-modified-id="有诱导-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>有诱导</a></span></li></ul></div>

# 组合预测

步骤：
    1.模型预测
        |
    2.模型组合
        |
    3.
        |
    4.

In [148]:
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers

class CombinationForecasting():
    '''
    组合预测模型
        普通组合预测：
            1.run() -->得到权重和最优值
            2.组合预测
        诱导组合预测：
            1.induce()  -->得到诱导矩阵
            2.run()  -->得到权重和最优值
            3.restoreW() -->得到还原后的权重矩阵
            4.对还原后的权重矩阵进行处理，得到用于预测的权重
            5.组合预测
    '''
    def induce(self, error, induceData):
        '''
        通过诱导值对误差值进行排序，请将两组数据顺序对应
        error --> result
        Args:
            error: 误差值
            induceData: 诱导值
        '''
        T = induceData.shape[0]
        Nc = induceData.shape[1]
        s1 = np.argsort(-induceData)
        result = []
        for i in range(T):
            eCol = error.iloc[i, s1.iloc[i, :]]
            result.append(eCol.tolist())

        return pd.DataFrame(result)
    
    def restoreW(self,w,result, induceData):
        '''
        诱导之后的权重矩阵(用于结果展示)
        result --> error
        '''
        T = induceData.shape[0]
        Nc = induceData.shape[1]
        s1 = np.argsort(-induceData)
        s2 = np.argsort(s1)
        Wmatrix = []
        for i in range(T):
            wCol = w[s2.iloc[i, :]]
            Wmatrix.append(wCol.tolist())
        Wmatrix = pd.DataFrame(Wmatrix,columns = induceData.columns)
        return Wmatrix
        
    def run(self,data, Nonnegative = True,errorType = 'mse',rounds = 4):
            '''
            组合预测model
            Args:
                data: 传入的误差数据，[e1, e2,..]
                Nonnegative: 是否有非负约束，默认有,只对mse优化模型有效
                errorType:
                ['mse','abse','bias','range']
                    mse:误差平方和最小
                    abse:绝对误差和最小
                    bias:最大偏差最小
                    range:误差极差最小
                rounds:保留小数的位数，默认4位
            return:
                返回各个预测方法的权重，w = [w1, w2, w3, ....]
            '''
            T = data.shape[0]
            Nc = data.shape[1]
            total = T * 2 + Nc
            errorMatrix = np.dot(np.transpose(data),data)
            if errorType == 'mse':
                ## 误差平方和最小
                ## 误差信息矩阵E
                E = np.dot(np.transpose(data),data)
                ## 优化目标
                Q = 2*matrix(np.array(E.tolist()))
                p = matrix([0.0 for i in range(Nc)])
                ## 非负约束
                G = matrix(np.negative(np.identity(Nc)).tolist())
                h = p
                ## 和为1
                A = matrix([1.0 for i in range(Nc)],(1, Nc))
                b = matrix(1.0)
                print('********误差平方和最小进行优化************')

                if Nonnegative==False:
                    sol=solvers.qp(Q, p,A=A,b = b)
                else:
                    sol=solvers.qp(Q, p,G=G,h=h,A=A,b= b)
                w = sol['x']
                primal_objective = sol['primal objective']

            elif errorType=='abse':
                ### 绝对误差和最小
                ## 优化目标
                c = matrix([1.0 for i in range(2*T)] + [0.0 for i in range(Nc)])
                ## 限制条件
                A1 = np.concatenate((np.negative(np.identity(T)),np.identity(T),data),axis = 1).tolist()
                A2 = [0.0 for i in range(2*T)] + [1.0 for i in range(Nc)]
                A1.append(A2)
                A = np.transpose(np.array(A1))
                A = matrix(A.tolist())
                b = matrix([0.0 for i in range(T)] + [1])
                ## 非负约束
                G = matrix(np.negative(np.identity(total)).tolist())
                h = matrix([0.0 for i in range(total)])
                print('**************绝对误差和最小进行优化********************')
                sol = solvers.lp(c,G, h, A, b)
                w = sol['x'][-Nc:]
                primal_objective = sol['primal objective']

            elif errorType == 'bias':
                ## 优化目标
                c = matrix([0.0 for i in range(total)] + [1.0])
                ## 限制条件
                A1 = np.concatenate((np.negative(np.identity(T)),np.identity(T),data,np.zeros(shape=(T,1))),axis = 1).tolist()
                A2 = [0.0 for i in range(2*T)] + [1.0 for i in range(Nc)] + [0]
                A1.append(A2)
                A = np.transpose(np.array(A1))
                A = matrix(A.tolist())
                b = matrix([0.0 for i in range(T)] + [1])
                ## 非负约束
                G1 = np.negative(np.identity(total + 1))
                # print(G1)
                G2 = np.concatenate((np.identity(T),np.identity(T),np.zeros(shape=(T,Nc)),np.negative(np.ones(shape=(T,1)))),axis = 1)
                G = np.transpose(np.concatenate((G1, G2)))
                G = matrix(G.tolist())
                h = matrix([0.0 for i in range(T +total + 1)])
                print('********************最大偏差最小进行优化************************')

                sol = solvers.lp(c,G, h, A, b)
                w = sol['x'][-Nc - 1: -1]
                primal_objective = sol['primal objective']

            elif errorType == 'range':
                ## 优化目标
                c = matrix([0.0 for i in range(total)] + [1.0, -1.0])
                ## 限制条件
                A = matrix([0.0 for i in range(2*T)]+[1.0 for i in range(Nc)] + [0.0,0.0],(1,total+2))
                b = matrix([1.0])
                ## 非负约束
                G1 = np.concatenate((np.negative(np.identity(total)),np.zeros(shape = (total, 2))),axis = 1)
                # print(G1)
                G2 = np.concatenate((np.zeros(shape = (T,2*T)),data,np.negative(np.ones(shape=(T,1))),np.zeros(shape=(T,1))),axis = 1)
                G3 = np.concatenate((np.zeros(shape = (T,2*T)),np.negative(data), np.zeros(shape=(T,1)),np.ones(shape=(T,1))),axis = 1)
                G = np.transpose(np.concatenate((G1, G2, G3)))
                G = matrix(G.tolist(),(2*T +total,total + 2))
                h = matrix([0.0 for i in range(2*T +total)])
                print('**************误差极差最小进行优化*********************')

                sol = solvers.lp(c,G, h, A, b)
                w = sol['x'][-Nc-2:-2]
                primal_objective = sol['primal objective']
                
            else:
                print("无该方法")
                
            return {'w':np.round(list(w), rounds),'primal_objective':round(primal_objective,rounds),'errorMatrix':errorMatrix}

## 无诱导

In [133]:
data = pd.read_excel(r'example/example1.xlsx')
### 传入的数据中，e*列判断为error,a*列判断为诱导值列（一般是精度）
errorCol = []
for i in data.columns[1:]:
    if 'e' in i:
        errorCol.append(i)
induceCol = []
for i in data.columns[1:]:
    if 'a' in i:
        induceCol.append(i)
error = data[errorCol]
inducdData = data[induceCol]


In [135]:
## 非诱导情况
types = ['mse','abse','bias','range']
cf = CombinationForecasting()
for t in types:
    res = cf.run(error, errorType = t)
    print(f'权重为:{res["w"]}')
    print(f'最优值为:{res["primal_objective"]}')
    print(f'误差信息矩阵为:\n{res["errorMatrix"]}')


********误差平方和最小进行优化************
     pcost       dcost       gap    pres   dres
 0:  1.3069e+07  1.3069e+07  6e+00  2e+00  3e+00
 1:  1.3069e+07  1.3069e+07  9e-01  3e-01  4e-01
 2:  1.3069e+07  1.3069e+07  8e-01  3e-01  4e-01
 3:  4.3888e+07 -5.3780e+06  5e+07  6e-02  9e-02
 4:  4.0808e+07 -4.2487e+07  9e+07  6e-02  8e-02
 5:  3.0445e+07 -1.5719e+08  2e+08  5e-02  8e-02
 6:  2.8091e+07 -2.2423e+07  5e+07  1e-02  2e-02
 7:  1.4271e+07 -5.3637e+07  7e+07  1e-02  2e-02
 8:  1.4334e+07  1.2969e+07  1e+06  2e-04  3e-04
 9:  1.4311e+07  1.4296e+07  2e+04  2e-06  4e-06
10:  1.4311e+07  1.4310e+07  2e+02  2e-08  4e-08
11:  1.4311e+07  1.4311e+07  2e+00  2e-10  3e-09
Optimal solution found.
权重为:[0.4685 0.5315 0.    ]
最优值为:14310532.62
误差信息矩阵:
[[34141800.279      -3169618.6454      8578997.0638    ]
 [-3169618.6454     29718306.4386     34355045.14049999]
 [ 8578997.0638     34355045.14049999 90845952.6447    ]]
**************绝对误差和最小进行优化********************
     pcost       dcost       gap    pr

## 有诱导

In [149]:
data = pd.read_excel(r'example/example2.xlsx')
### 传入的数据中，e*列判断为error,a*列判断为诱导值列（一般是精度）
errorCol = []
for i in data.columns[1:]:
    if 'e' in i:
        errorCol.append(i)
induceCol = []
for i in data.columns[1:]:
    if 'a' in i:
        induceCol.append(i)
error = data[errorCol]
inducdData = data[induceCol]

In [159]:
types = ['mse','abse','bias','range']
dic = {'mse':'误差平方和最小进行优化',
       'abse':'绝对误差和最小进行优化',
       'bias':'最大偏差最小进行优化',
       'range':'误差极差最小进行优化'}
cf = CombinationForecasting()
result = cf.induce(error, induceData)
with open('output/result.txt', mode = 'w+') as fp:
    for t in types:
        res = cf.run(result, errorType = t)
        Nowtime = time.strftime('%Y-%m-%d  %H:%M:%S',time.localtime(time.time()))
        fp.write(f'-----------{Nowtime}----------\n')
        fp.write(f'-----------{dic[t]}----------\n')
        print(f'权重为:{res["w"]}')
        fp.write(f'权重为:{res["w"]}\n')
        print(f'最优值为:{res["primal_objective"]}')
        fp.write(f'最优值为:{res["primal_objective"]}\n')
        print(f'误差信息矩阵为:\n{res["errorMatrix"]}')
        fp.write(f'误差信息矩阵为:\n{res["errorMatrix"]}\n')
        print(f'权重矩阵为:\n{cf.restoreW(w, error, induceData)}')
        fp.write(f'权重矩阵为:\n{cf.restoreW(w, error, induceData)}\n')

********误差平方和最小进行优化************
     pcost       dcost       gap    pres   dres
 0:  2.6213e+08  2.6213e+08  1e+00  3e-16  3e+00
 1:  2.6213e+08  2.6213e+08  1e-02  3e-17  3e-02
 2:  2.6213e+08  2.6213e+08  1e-04  2e-16  3e-04
 3:  2.6213e+08  2.6213e+08  1e-06  2e-16  3e-06
 4:  2.6213e+08  2.6213e+08  1e-08  1e-16  5e-07
 5:  2.6213e+08  2.6213e+08  1e-10  6e-17  5e-07
 6:  2.6213e+08  2.6213e+08  1e-12  1e-16  5e-07
 7:  2.6213e+08  2.6213e+08  1e-14  6e-17  5e-07
 8:  2.6213e+08  2.6213e+08  1e-16  3e-17  5e-07
 9:  2.6213e+08  2.6213e+08  1e-18  1e-16  5e-07
10:  2.6213e+08  2.6213e+08  1e-20  3e-17  5e-07
11:  2.6213e+08  2.6213e+08  1e-22  3e-17  5e-07
12:  2.6213e+08  2.6213e+08  1e-24  1e-16  5e-07
13:  2.6213e+08  2.6213e+08  1e-26  3e-17  5e-07
14:  2.6213e+08  2.6213e+08  1e-28  1e-16  5e-07
15:  2.6213e+08  2.6213e+08  1e-30  4e-17  5e-07
16:  2.6213e+08  2.6213e+08  1e-32  1e-16  5e-07
17:  2.6213e+08  2.6213e+08  1e-34  1e-16  5e-07
18:  2.6213e+08  2.6213e+08  1e-36  1e